In [27]:
import pandas as pd
import numpy as np
import re
import unicodedata

In [120]:
data = pd.read_csv('Datos Madrid/Datos Madrid.csv')

### Eliminación de registros sin información

En caso de presentar valores desconocidos en los campos Comentario, Mas detalles e Info. Será imposible
trabajar con esos registros, ya que no hay forma de obtener información de ellos.
Estos registros suelen corresponder a anuncios retirados que ya no disponían de información
a la hora de relaizar el raspado.

In [121]:
# Comprobamos los datos que cumplen esta condición

columns_to_study = ['Comentario', 'Mas detalles', 'Info']

incomplete_condition = (data[columns_to_study] == 'Desconocido').all(axis = 1)
data_incomplete = data[incomplete_condition]

In [122]:
print(data_incomplete.sample(8),'\n\n') # Comprobamos que están incompletos los campos

print('Cantidad de datos inservibles: ',len(data_incomplete)) # Visualizamos el total de datos incompletos

             ID       Precio Planta o Tipo N Habitaciones           m2  \
18957  93612281  Desconocido   Desconocido    Desconocido  Desconocido   
13125  97250208  Desconocido   Desconocido    Desconocido  Desconocido   
19184  96512946  Desconocido   Desconocido    Desconocido  Desconocido   
10154  97149248  Desconocido   Desconocido    Desconocido  Desconocido   
19002  92614545  Desconocido   Desconocido    Desconocido  Desconocido   
10196  96057949  Desconocido   Desconocido    Desconocido  Desconocido   
3623   97124127  Desconocido   Desconocido    Desconocido  Desconocido   
16366  95839808  Desconocido   Desconocido    Desconocido  Desconocido   

        Comentario                Zona            Subzona         Info  \
18957  Desconocido          Villaverde        Los Rosales  Desconocido   
13125  Desconocido         Carabanchel       Vista Alegre  Desconocido   
19184  Desconocido          Villaverde        Los Ángeles  Desconocido   
10154  Desconocido            Chamber

In [123]:
# Guardamos los valores completos en data2

data = data[~(incomplete_condition)]

In [124]:
len(data)

19351

In [125]:
data = data.reset_index(drop=True)

### Tratamiento de campo Planta o Tipo

In [126]:
# Comprobamos cuántos valores desconocidos hay en esta columna

sum(data['Planta o Tipo'] == 'Desconocido')

2555

In [127]:
def standard_words(sentence):
    return str(unicodedata.normalize('NFKD', sentence.lower()).encode('ASCII', 'ignore'))

def search_words(sentence, words_searched : list):
    return any([word in sentence for word in words_searched])

def search_words_in_df(df : pd.DataFrame, columns_to_study : list, words_searched : list):
    return df[columns_to_study].applymap(lambda x: search_words(x,words_searched)).any(axis = 1)

def is_any_value_filled(df : pd.DataFrame, list_filled : list, column_compare, value_null):
    return sum(list_filled & (df[column_compare] == value_null))

In [128]:
sum(search_words_in_df(data,['Comentario'],['1º','1ª']))

428

In [133]:
def apply_conditions(data, condition_list, name_fill, column_compare = 'Planta o Tipo', value_null = 'Desconocido'):
    condition_under = np.array(condition_list).reshape((len(condition_list),-1)).any(axis=0)
    values_filled = is_any_value_filled(data, condition_under, column_compare, value_null)
    print(f'Cantidad de valores rellenados con valor {name_fill}: {values_filled}')
    condition_under = condition_under & (data[column_compare]==value_null)
    data.loc[condition_under, column_compare] = name_fill
    return data

def search_number(sentence):
    match = re.search('planta[^0-9]{0,3}(\d{1,})', sentence)
    if match is None:
        return 'Desconocido'
    else:
        return match[1]

In [131]:
data[['Comentario_lower', 'Mas detalles_lower', 'Info_lower']] = data[['Comentario', 'Mas detalles', 'Info']].applymap(lambda x: standard_words(x))

In [116]:
condition_list = []
words_searched = ['chalet', 'finca', 'casa rural']

condition_list.append(search_words_in_df(data, ['Comentario_lower', 'Mas detalles_lower', 'Info_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Chalet')

Cantidad de valores rellenados con valor Chalet: 1415


In [117]:
# Comprobamos cuántos valores desconocidos nos quedan ahora

sum(data['Planta o Tipo'] == 'Desconocido')

1140

In [118]:
np.unique(data['Planta o Tipo'], return_counts=True)

# Vemos que hay valores con -, quiere decir que no cargó el valor numérico de Info al presentar un guión de por medio

(array(['-', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'Bajo', 'Chalet',
        'Desconocido'], dtype=object),
 array([  26, 4051, 3212, 2483, 2008, 1132,  723,  392,  216,  130, 2423,
        1415, 1140]))

In [140]:
# Correccion de errores en la toma de numeros de planta en el scrapeo
data['Planta o Tipo'] = data['Info_lower'].apply(lambda x: search_number(x))
print(np.unique(data['Planta o Tipo'], return_counts=True))

(array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '20', '22', '3', '4', '5', '6', '60', '7', '8', '9',
       'Desconocido'], dtype=object), array([3821,   76,   53,   28,   43,   20,   13,   10,    1,    7,    3,
       3203,    9,    2, 2483, 2008, 1132,  722,    1,  392,  216,  130,
       4978]))


In [43]:
# Otro tipo de inmueble habitual y característico serían las entreplantas, observando  el campo Info
# y Mas detalles, se aprecia que, para este tipo de propiedades, se suele detallar en estos campos
# Siguiendo un proceso similar a los anteriores:
condition_list = []
words_searched = ['este local comercial', 'de uso comercial', 'es local comercial', 'no acondicionado', 
                  'para acondicionar', 'a acondicionar', 'de oficinas', 'uso industrial', 'local industrial']

condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))


words_searched = ['Local comercial']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))

data = apply_conditions(data, condition_list, 'Uso comercial')

Cantidad de valores rellenados con valor Uso comercial: 10


In [44]:
condition_list = []
words_searched = ['luminosa buhardilla', 'preciosa buhardilla', 'bonita buhardilla', 'luminoso atico', 
                  'bonito atico', 'precioso atico', 'luminosa azotea', 'preciosa azotea', 'excepcional atico', 
                  'excepcional atico', 'bonita azotea', 'azotea en venta', 'azotea a la venta', 'se vende azotea', 
                  'atico en venta', 'atico a la venta', 'se vende atico', 'buhardilla en venta', 'buhardilla a la venta', 
                  'se vende buhardilla', 'venta de atico', 'venta de azotea', 'piso azotea', 'piso atico', 
                  'venta de buhardilla']

condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))

words_searched = ['Atico', 'Azotea', 'Buhardilla', 'Ultima planta']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))

words_searched = ['azotea', 'atico']
condition_list.append(search_words_in_df(data, ['Info_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Atico')

Cantidad de valores rellenados con valor Atico: 7


In [45]:
# Otro tipo de inmueble habitual y característico serían las entreplantas, observando  el campo Info
# y Mas detalles, se aprecia que, para este tipo de propiedades, se suele detallar en estos campos
# Siguiendo un proceso similar a los anteriores:
condition_list = []
words_searched = ['en entreplanta', 'en la entreplanta', 'esta entreplanta', 'piso en entreplanta', 'inmueble en entreplanta',
                 'se trata de una entreplanta', 'en una entreplanta']

condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))


words_searched = ['entreplanta']
condition_list.append(search_words_in_df(data, ['Info_lower', 'Mas detalles_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Entreplanta')

Cantidad de valores rellenados con valor Entreplanta: 204


In [46]:
condition_list = []
words_searched = ['vende sotano', 'vende semisotano', 'es un sotano', 'es un bajo', 'es el bajo', 'se trata de un bajo', 
                'es un semisotano', 'vende un semisotano', 'vende un sotano', 'es una planta baja', 'es la planta baja', 
                'es planta baja', 'en planta baja', 'es sotano', 'es semisotano', 'es el sotano', 'es el semisotano',
                'en el sotano', 'en el semisotano', 'en la planta baja', 'semisotano a la venta', 'semisotano en venta', 
                 'sotano a la venta', 'sotano en venta', 'semisotano exterior', 'sotano exterior', 'planta baja exterior', 
                'bonito sotano', 'estupendo sotano', 'precioso sotano', 'magnifico sotano', 'en un semisotano', 
                'en planta sotano', 'semi sotano', 'en un sotano', 'en una planta baja', 'en planta semisotano', 
                'en la planta sotano', 'en la planta semisotano', 'piso en planta baja', 'vivienda en planta baja', 
                'inmueble en sotano', 'piso en sotano', 'vivenda en sotano', 'inmueble en semisotano', 'piso en semisotano', 
                'vivenda en semisotano', 'inmueble en semi sotano', 'piso en semi sotano', 'vivenda en semi sotano', 
                'inmueble en semi-sotano', 'piso en semi-sotano', 'vivenda en semi-sotano']

condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))


words_searched = ['Bajo', 'Planta baja', 'Sotano', 'Semisotano', 'Semi-sotano', 'esta en la planta baja', 
                  'Ubicada en la planta baja']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))

words_searched = ['semisotano', 'semi sotano', 'semi-sotano']
condition_list.append(search_words_in_df(data, ['Info_lower', 'Mas detalles_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Bajo') 

Cantidad de valores rellenados con valor Bajo: 213


In [47]:
condition_list = []
words_searched = ['apartamento', 'estudio', 'piso en venta', 'piso a la venta', 'vende piso', 
                'estupendo piso', 'precioso piso', 'espacioso piso', 'esplendido piso', 'luminoso piso', 
                'fantastico piso', 'excelente piso', 'magnifico piso', 'fabuloso piso', 'comodo piso', 
                'amplio piso', 'piso estupendo', 'piso precioso', 'piso espacioso', 'piso esplendido', 
                'piso luminoso ', 'piso fantastico ', 'piso perfecto ', 'piso excelente', 'piso magnifico', 
                'fabuloso piso', 'piso comodo', 'piso amplio', 'piso maravilloso ', 'piso exterior', 'piso interior', 
                'piso reformado', 'piso con reforma', 'piso a reformar', 'loft', 'duplex', 'triplex', 'piso de', 
                'piso ubicado', 'piso situado', 'lujoso piso', 'piso totalmente', 'piso en ', 'piso piloto', 'piso muy', 
                'piso con', 'piso situado', 'piso reformado', 'piso sin reformar', 'piso a reformar', 'piso para reformar', 
                'piso a falta de reformar', 'piso bastante']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Piso') 


Cantidad de valores rellenados con valor Piso: 505


In [48]:
condition_list = []
words_searched = ['viviendas', 'complejo residencial', 'inmuebles', 'pisos', 'apartamentos']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))

words_searched = ['estudio']
condition_list.append(search_words_in_df(data, ['Info_lower', 'Mas detalles_lower'], words_searched))
data = apply_conditions(data, condition_list, 'Residencial') 

Cantidad de valores rellenados con valor Residencial: 86


In [49]:
condition_list = []
words_searched = ['primera planta', 'segunda planta', 'tercera planta', 'cuarta planta', 
                 'quinta planta', 'sexta planta', 'septima planta', 'octava planta', 
                 'novena planta', 'decima planta', 'planta primera', 'planta segunda', 
                 'planta tercera', 'planta cuarta', 'planta quinta', 'planta sexta', 
                 'planta septima', 'planta octava', 'primer piso', 'segundo piso', 'tercer piso', 
                 'cuarto piso', 'quinto piso', 'sexto piso', 'septimo piso', 'octavo piso', 
                 'noveno piso', 'decimo piso', 'con ascensor', 'de ascensor']

condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))

words_searched = ['ascensor']

condition_list.append(search_words_in_df(data, ['Mas detalles_lower'], words_searched))

data = apply_conditions(data, condition_list, 'Piso') 

Cantidad de valores rellenados con valor Piso: 107


In [50]:
# Comprobamos que ya no quedan valores desconocidos

data[data['Planta o Tipo'] == 'Desconocido']

,ID,Precio,Planta o Tipo,N Habitaciones,m2,Comentario,Zona,Subzona,Info,Mas detalles,Comentario_lower,Mas detalles_lower,Info_lower
3303,94618811,885.300,Desconocido,4,204,"[Vivienda ubicada en Mirasierra, zona exclusiv...",Fuencarral,Mirasierra,Obra nueva 204 m² 4 hab.,"[204 m² construidos, 159 m² útiles, 4 habitaci...","b'[vivienda ubicada en mirasierra, zona exclus...","b'[204 m2 construidos, 159 m2 utiles, 4 habita...",b' obra nueva 204 m2 4 hab. '
3334,97267269,1.790.000,Desconocido,6,690,[Vivienda unifamiliar aislada. El inmueble se ...,Fuencarral,Mirasierra,690 m² 6 hab.,"[690 m² construidos, 546 m² útiles, 6 habitaci...",b'[vivienda unifamiliar aislada. el inmueble s...,"b'[690 m2 construidos, 546 m2 utiles, 6 habita...",b' 690 m2 6 hab. '
9763,91232758,278.000,Desconocido,3,89,"[Vivienda de 88,77 metros cuadrados, ubicada e...",Barrio de Salamanca,Guindalera,89 m² 3 hab.,"[89 m² construidos, 78 m² útiles, 3 habitacion...","b'[vivienda de 88,77 metros cuadrados, ubicada...","b'[89 m2 construidos, 78 m2 utiles, 3 habitaci...",b' 89 m2 3 hab. '
12053,94964414,115.000,Desconocido,3,40,[. Oportunidad de inversión: pendiente de Pose...,Latina,Puerta del Ángel,40 m² 3 hab.,"[40 m² construidos, 36 m² útiles, 3 habitacion...",b'[. oportunidad de inversion: pendiente de po...,"b'[40 m2 construidos, 36 m2 utiles, 3 habitaci...",b' 40 m2 3 hab. '
15441,97236354,135.000,Desconocido,3,57,[Dispone de una superficie de 57 m² distribuid...,Puente de Vallecas,San Diego,57 m² 3 hab.,"[57 m² construidos, 3 habitaciones, 1 baño, Se...",b'[dispone de una superficie de 57 m2 distribu...,"b'[57 m2 construidos, 3 habitaciones, 1 bano, ...",b' 57 m2 3 hab. '
15473,96924113,154.500,Desconocido,2,72,[¿Quieres vivir en una zona consolidada con pa...,Puente de Vallecas,San Diego,72 m² 2 hab.,"[72 m² construidos, 2 habitaciones, 1 baño, Se...",b'[quieres vivir en una zona consolidada con p...,"b'[72 m2 construidos, 2 habitaciones, 1 bano, ...",b' 72 m2 2 hab. '
16353,94347647,121.000,Desconocido,2,47,[Vivienda de 2 dormitorios en edificio sin asc...,Puente de Vallecas,Portazgo,47 m² 2 hab.,"[47 m² construidos, 41 m² útiles, 2 habitacion...",b'[vivienda de 2 dormitorios en edificio sin a...,"b'[47 m2 construidos, 41 m2 utiles, 2 habitaci...",b' 47 m2 2 hab. '
17141,97306679,104.617,Desconocido,3,48,"[, Vivienda situada en edificio residencial de...",Vicalvaro,Casco Histórico de Vicálvaro,48 m² 3 hab.,"[48 m² construidos, 3 habitaciones, 1 baño, Se...","b'[, vivienda situada en edificio residencial ...","b'[48 m2 construidos, 3 habitaciones, 1 bano, ...",b' 48 m2 3 hab. '


## Tratamiento de plantas con nuevo campo piso

Creamos un nuevo campo Planta para detallar la altura de la planta del piso en oferta, dejaremos el campo Planta o Tipo como Tipo

In [135]:
# Creamos un nuevo dataset llamado df y renombramos la columna Planta o Tipo por solamente Tipo

data.rename(columns={'Planta o Tipo': 'Tipo'}, inplace=True)

In [52]:
# Creamos la nueva columna Piso con valores cero por defecto

data['Planta'] = data['Tipo'].apply(lambda x: x if re.search('[0-9]*)', sentence) is not None else "0")
data['Tipo'] = data['Tipo'].apply(lambda x: "Piso" if x in lista_numericos else x)

In [31]:
# Buscamos palabras y expresiones habituales para los cada piso


#### PLANTA 1 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['primera planta', '1a planta', 'planta 1 ', 'piso 1 ', '1er piso', 'primer piso', 
                  'planta primera', 'piso primero', 'un primer', 'un primero', 
                  'el primer', 'el primer', 'una primera', 'la primera', '1a', '1o', 'un primero']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '1', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['1ª', '1º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '1', 'Planta', '0')

### PLANTA 2 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['segunda planta' , '2a planta', '2o piso', 'planta 2 ', 'piso 2 ' , 
                  'segundo piso', 'planta segunda', 'piso segundo', 'un segundo', 
                  'el segundo', 'una segunda', 'la segunda', '2o', '2a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '2', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['2ª', '2º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '2', 'Planta', '0')

### PLANTA 3 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['tercera planta', '3a planta', '3o piso', 'planta 3 ', 'piso 3 ', 'tercer piso', 'planta tercera', 'piso tercero', 'un tercer', 'el tercer', 'el tercero', 'una tercera', 'la tercera', '3o', '3a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '3', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['3ª', '3º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '3', 'Planta', '0')

### PLANTA 4 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['cuarta planta' , '4a planta', '4o piso', 'planta 4 ', 'piso 4 ' , 'cuarto piso', 'planta cuarta', 'piso cuarto', 'un cuarto', 'el cuarto', 'una cuarta', 'la cuarta', '4o', '4a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '4', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['4ª', '4º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '4', 'Planta', '0')

### PLANTA 5 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['quinta planta' , '5a planta', '5o piso', 'planta 5 ', 'piso 5 ' , 'quinto piso', 'planta quinta', 'piso quinto', 'un quinto', 'el quinto', 'una quinta', 'la quinta', '5o', '5a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '5', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['5ª', '5º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '5', 'Planta', '0')

### PLANTA 6 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['sexta planta' , '6a planta', '6o piso' , 'sexto piso', 'planta sexta', 'piso sexto', 'un sexto', 'el sexto', 'una sexta', 'la sexta', '6o', '6a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '6', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['6ª', '6º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '6', 'Planta', '0')

### PLANTA 7 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['septima planta' , '7a planta', '7o piso', 'planta 7 ', 'piso 7 ', 'septimo piso', 'planta septima', 'piso septimo', 'un septimo', 'el septimo', 'una septima', 'la septima', '7o', '7a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '7', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['7ª', '7º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '7', 'Planta', '0')

### PLANTA 8 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['octava planta' , '8a planta', '8o piso', 'planta 8 ', 'piso 8 ' , 'octavo piso', 'planta octava', 'piso octavo', 'un octavo', 'el octavo', 'una octava', 'la octava', '8o', '8a' ]
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '8', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['8ª', '8º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '8', 'Planta', '0')

### PLANTA 9 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['novena planta' , '9a planta', '9o piso', 'planta 9 ', 'piso 9 ', 'noveno piso', 'planta novena', 'piso noveno', 'un noveno', 'el noveno', 'una novena', 'la novena', '9o', '9a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '9', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['9ª', '9º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '9', 'Planta', '0')

### PLANTA 10 ####
# Búsqueda en campo comentario en minúsculas ignorando mayúsculas
condition_list = []
words_searched = ['decima planta' , '10a planta', '10 piso', 'planta 10 ', 'piso 10 ' , 'decimo piso', 'planta decima', 'piso decimo', 'un decimo', 'el decimo', 'una decima', 'la decima', '10o', '10a']
condition_list.append(search_words_in_df(data, ['Comentario_lower'], words_searched))
data = apply_conditions(data, condition_list, '10', 'Planta', '0')

# Búsqueda en campo comentario normal, con signos literales del anuncio
condition_list = []
words_searched = ['10ª', '10º']
condition_list.append(search_words_in_df(data, ['Comentario'], words_searched))
data = apply_conditions(data, condition_list, '10', 'Planta', '0')

Cantidad de valores rellenados con valor 1: 854
Cantidad de valores rellenados con valor 1: 0
Cantidad de valores rellenados con valor 2: 234
Cantidad de valores rellenados con valor 2: 0
Cantidad de valores rellenados con valor 3: 80
Cantidad de valores rellenados con valor 3: 0
Cantidad de valores rellenados con valor 4: 172
Cantidad de valores rellenados con valor 4: 0
Cantidad de valores rellenados con valor 5: 30
Cantidad de valores rellenados con valor 5: 0
Cantidad de valores rellenados con valor 6: 6
Cantidad de valores rellenados con valor 6: 0
Cantidad de valores rellenados con valor 7: 6
Cantidad de valores rellenados con valor 7: 0
Cantidad de valores rellenados con valor 8: 3
Cantidad de valores rellenados con valor 8: 0
Cantidad de valores rellenados con valor 9: 0
Cantidad de valores rellenados con valor 9: 0
Cantidad de valores rellenados con valor 10: 3
Cantidad de valores rellenados con valor 10: 0


In [32]:
data[(data['Planta'] == '0') & (data['Tipo'] == 'Piso')] 

,ID,Precio,Tipo,N Habitaciones,m2,Comentario,Zona,Subzona,Info,Mas detalles,Comentario_lower,Mas detalles_lower,Info_lower,Planta
84,85449897,579.000,Piso,5,131,[Vivienda2 os brinda la oportunidad de inverti...,Centro,Malasaña-Universidad,131 m² 5 hab. exterior con ascensor,"[131 m² construidos, 117 m² útiles, 5 habitaci...",b'[vivienda2 os brinda la oportunidad de inver...,"b'[131 m2 construidos, 117 m2 utiles, 5 habita...",b' 131 m2 5 hab. exterior con ascensor ',0
111,97322351,109.900,Piso,Desconocido,27,[Redpiso pone a la venta Estudio ACONDICIONADO...,Centro,Malasaña-Universidad,27 m² exterior sin ascensor,"[27 m² construidos, Sin habitación, 1 baño, Se...",b'[redpiso pone a la venta estudio acondiciona...,"b'[27 m2 construidos, sin habitacion, 1 bano, ...",b' 27 m2 exterior sin ascensor ',0
153,97286762,156.000,Piso,1,29,"[Redpiso San Bernardo, vende acogedor apartame...",Centro,Malasaña-Universidad,29 m² 1 hab. sin ascensor,"[29 m² construidos, 1 habitación, 1 baño, Segu...","b'[redpiso san bernardo, vende acogedor aparta...","b'[29 m2 construidos, 1 habitacion, 1 bano, se...",b' 29 m2 1 hab. sin ascensor ',0
245,95895760,685.000,Piso,3,103,[Piso reformado a estrenar Engel&amp;Voelkers ...,Centro,Malasaña-Universidad,103 m² 3 hab. exterior sin ascensor,"[103 m² construidos, 93 m² útiles, 3 habitacio...",b'[piso reformado a estrenar engel&amp;voelker...,"b'[103 m2 construidos, 93 m2 utiles, 3 habitac...",b' 103 m2 3 hab. exterior sin ascensor ',0
253,95806565,490.000,Piso,2,74,[Vivienda Reformada en Malasaña Engel&amp;Voel...,Centro,Malasaña-Universidad,74 m² 2 hab. exterior con ascensor,"[74 m² construidos, 57 m² útiles, 2 habitacion...",b'[vivienda reformada en malasana engel&amp;vo...,"b'[74 m2 construidos, 57 m2 utiles, 2 habitaci...",b' 74 m2 2 hab. exterior con ascensor ',0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19284,96323505,466.500,Piso,2,87,"[Magnífico piso de obra nueva en Madrid, perfe...",Hortaleza,Virgen del Cortijo - Manoteras,Obra nueva 87 m² 2 hab.,"[87 m² construidos, 76 m² útiles, 2 habitacion...","b'[magnifico piso de obra nueva en madrid, per...","b'[87 m2 construidos, 76 m2 utiles, 2 habitaci...",b' obra nueva 87 m2 2 hab. ',0
19285,93589718,433.000,Piso,2,87,"[Fantástico piso de obra nueva en Madrid, idea...",Hortaleza,Virgen del Cortijo - Manoteras,Obra nueva 87 m² 2 hab.,"[87 m² construidos, 76 m² útiles, 2 habitacion...","b'[fantastico piso de obra nueva en madrid, id...","b'[87 m2 construidos, 76 m2 utiles, 2 habitaci...",b' obra nueva 87 m2 2 hab. ',0
19286,94211075,497.000,Piso,2,100,"[Fabuloso piso de obra nueva en Madrid, perfec...",Hortaleza,Virgen del Cortijo - Manoteras,Obra nueva 100 m² 2 hab.,"[100 m² construidos, 85 m² útiles, 2 habitacio...","b'[fabuloso piso de obra nueva en madrid, perf...","b'[100 m2 construidos, 85 m2 utiles, 2 habitac...",b' obra nueva 100 m2 2 hab. ',0
19295,95613429,450.000,Piso,2,180,[Colibree presenta asombroso loft reformado in...,Hortaleza,Virgen del Cortijo - Manoteras,180 m² 2 hab. exterior sin ascensor G...,"[180 m² construidos, 2 habitaciones, 2 baños, ...",b'[colibree presenta asombroso loft reformado ...,"b'[180 m2 construidos, 2 habitaciones, 2 banos...",b' 180 m2 2 hab. exterior sin ascensor ...,0


## Creación de campo número de plantas

Alguno de los pisos que se ofertan poseen en el campo Comentario indicaciones de que se trata de un duplex o triplex en algún caso, guardaremos este dato y lo añadiremos en un campo Cantidad de plantas. Este campo será 1 para todos los pisos excepto aquellos que posean más.

In [ ]:
data_piso = df3[(df3['Tipo'] == 'Piso') & ]

In [ ]:
# Buscamos pisos de nuevo, la mayoría de restantes serán de este tipo
# si incluyen el número de planta en el comentario, debería de tratarse de un piso

val_mas_plantas = []

for i in range(len(df3)):
    # Convertimos todo a minúsculas
    comentario_actual = df3['Comentario'].iloc[i].lower()
    mas_detalles_actual = df3['Mas detalles'].iloc[i].lower()
    # Ignoramos tildes, ñ y ç
    comentario_actual = unicodedata.normalize('NFKD', comentario_actual).encode('ASCII', 'ignore')
    mas_detalles_actual = unicodedata.normalize('NFKD', mas_detalles_actual).encode('ASCII', 'ignore')
    
    # Frases habituales de venta de este tipo de viviendas
    cond_duplex = ('duplex' in str(comentario_actual) or
                      'vivienda de dos plantas' in str(comentario_actual) or
                      'se compone de dos plantas' in str(comentario_actual) or
                      'vivienda de dos pisos' in str(comentario_actual) or
                      'se compone de dos pisos' in str(comentario_actual) or
                      'chalé de dos pisos' in str(comentario_actual) or
                      'finca de dos pisos' in str(comentario_actual) or
                      'chalé de dos plantas' in str(comentario_actual) or
                      'finca de dos plantas' in str(comentario_actual))
    
    cond_triplex = ('triplex' in str(comentario_actual) or
                      'vivienda de tres plantas' in str(comentario_actual) or
                      'se compone de tres plantas' in str(comentario_actual) or
                      'vivienda de tres pisos' in str(comentario_actual) or
                      'se compone de tres pisos' in str(comentario_actual) or
                      'chalé de tres pisos' in str(comentario_actual) or
                      'finca de tres pisos' in str(comentario_actual) or
                      'chalé de tres plantas' in str(comentario_actual) or
                      'finca de tres plantas' in str(comentario_actual))
    
    
    
    if cond_duplex == True:
        val_mas_plantas.append(2)
        cond_duplex = False
        
    elif cond_triplex == True:
        val_mas_plantas.append(3)
        cond_triplex = False
        
    else:
        val_mas_plantas.append(1)    
   

In [ ]:
# Los valores de este resto no parecen presentar datos del número de la planta en que se encuentran, por lo que podremos
# cubrir los faltantes con el resto:

df4 = df3.copy()

df4['Cantidad de plantas'] = val_mas_plantas

In [ ]:
df4[df4['Cantidad de plantas']!=1]

,ID,Precio,Tipo,N Habitaciones,m2,Comentario,Zona,Subzona,Info,Mas detalles,Planta,Cantidad de plantas
52,92409370,968.198,Piso,2,125,[Piso de dos dormitorios exterior en edificio ...,Centro,Malasaña-Universidad,125 m² 2 hab. Planta 3ª exterior con asc...,"[125 m² construidos, 2 habitaciones, 2 baños, ...",3,2
129,1850833,315.000,Piso,2,52,[Se vende piso dúplex abuhardillado en una de ...,Centro,Malasaña-Universidad,52 m² 2 hab. Planta 5ª interior con asce...,"[52 m² construidos, 46 m² útiles, 2 habitacion...",5,2
133,96335949,2.990.000,Piso,3,383,"[GLOBAL REAL ESTATE, le ofrece en zona centro ...",Centro,Malasaña-Universidad,383 m² 3 hab. Planta 10ª exterior con as...,"[383 m² construidos, 3 habitaciones, 4 baños, ...",10,2
175,96385204,168.000,Piso,1,30,[Ático Duplex. Muy luminoso. Ubicado en el cen...,Centro,Malasaña-Universidad,30 m² 1 hab. Planta 4ª interior sin asce...,"[30 m² construidos, 28 m² útiles, 1 habitación...",4,2
249,97053425,315.000,Piso,2,52,[Fantástico dúplex abuhardillado de 2 habitaci...,Centro,Malasaña-Universidad,52 m² 2 hab. Planta 5ª interior con asce...,"[52 m² construidos, 46 m² útiles, 2 habitacion...",5,2
...,...,...,...,...,...,...,...,...,...,...,...,...
19314,95869877,599.000,Piso,4,172,"[Pinar de Chamartín, piso dúplex, 4 dormitorio...",Hortaleza,Apóstol Santiago,172 m² 4 hab. Planta 2ª exterior con asc...,"[172 m² construidos, 127 m² útiles, 4 habitaci...",2,2
19322,96862805,350.000,Bajo,1,93,[Exclusivo duplex en urbanización privada junt...,Hortaleza,Apóstol Santiago,93 m² 1 hab. Bajo exterior con ascensor,"[93 m² construidos, 75 m² útiles, 1 habitación...",0,2
19324,93551334,350.000,Bajo,1,93,[Exclusivo duplex en urbanización privada junt...,Hortaleza,Apóstol Santiago,93 m² 1 hab. Bajo exterior con ascensor,"[93 m² construidos, 1 habitación, 2 baños, Seg...",0,2
19328,96519541,199.000,Piso,2,73,[DUPLEX 2 DORMITORIOS. Vivienda de 2 dormitori...,Hortaleza,Apóstol Santiago,73 m² 2 hab. Planta 1ª exterior sin asce...,"[73 m² construidos, 64 m² útiles, 2 habitacion...",1,2
